In [ ]:
pip install langchain langgraph google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
!pip install --quiet langchain langgraph langchain_community openai google-api-python-client google-auth google-auth-oauthlib


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your token.json here


In [ ]:
!pip install -U langchain-google-community


In [ ]:
!pip install -U langchain-google-community --upgrade
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import Tool

import os

# Replace with your actual values
os.environ["GOOGLE_CSE_ID"] = "CSE-ID-HERE"
os.environ["GOOGLE_API_KEY"] = "Your-API key-HERE"
search = GoogleSearchAPIWrapper()
search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
# Install the necessary library for Google Generative AI
!pip install -U langchain-google-genai


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_community import GoogleSearchAPIWrapper, GoogleSearchRun  # Correct import for newer version
from langgraph.graph import StateGraph
from typing import TypedDict, Literal, Union
from langchain.tools import tool
from googleapiclient.discovery import build
from google.oauth2 import service_account
import os

# Assuming GOOGLE_CSE_ID and GOOGLE_API_KEY are set elsewhere or defined here
# Replace with your actual values if not already set
os.environ["GOOGLE_CSE_ID"] = "CSE-ID-Here"
os.environ["GOOGLE_API_KEY"] = "Your-API key-Here"

# === Google Search Tool ===
# Ensure that the environment variables are properly set
search_wrapper = GoogleSearchAPIWrapper()  # This works with the newer 'langchain-google-community'
search_tool = GoogleSearchRun(api_wrapper=search_wrapper)  # Now passing the wrapper correctly

# === Google Calendar Tool (Custom) ===
# Use service account credentials for backend authentication
SERVICE_ACCOUNT_FILE = 'inbound-dahlia-459707-m5-ed416da1fa41.json'
SCOPES = ['https://www.googleapis.com/auth/calendar']

# Authenticate using service account
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_service = build('calendar', 'v3', credentials=creds)

@tool
def calendar_tool(query: str) -> str:
    """Creates a calendar event.""" # Added a docstring here
    # This is a placeholder; you'll need to parse the query to create dynamic events
    # based on the 'query' input. For now, it creates a fixed event as before.
    event = {
        'summary': 'Test Event',
        'start': {'dateTime': '2025-05-15T10:00:00+05:00'},
        'end': {'dateTime': '2025-05-15T11:00:00+05:00'},
    }
    try:
        created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
        return f"Calendar event created: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating calendar event: {e}"

# === Chat State ===
class ChatState(TypedDict):
    query: str
    tool_choice: Literal["search", "calendar", "chat"]
    result: Union[str, None]

# === Language Model ===
# Use ChatGoogleGenerativeAI with the appropriate Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# === Router ===
def route_tool(state: ChatState) -> ChatState:
    prompt = f"""You are a smart router. Classify the user query into one of the following:
    1. 'search' for web queries (e.g., 'Who won the match yesterday?')
    2. 'calendar' for events (e.g., 'Create a meeting tomorrow at 5 PM')
    3. 'chat' for general AI conversation

    Query: "{state['query']}"
    Answer with one word only: search, calendar, or chat.
    """
    # Use .invoke() with ChatGoogleGenerativeAI
    decision = llm.invoke(prompt).content.strip().lower()
    return {"query": state["query"], "tool_choice": decision, "result": None}

# === Tool Nodes ===
def use_search(state: ChatState) -> ChatState:
    print("Routing to Search...") # Added for debugging
    result = search_tool.run(state["query"])
    return {"query": state["query"], "tool_choice": "search", "result": result}

def use_calendar(state: ChatState) -> ChatState:
    print("Routing to Calendar...") # Added for debugging
    # Note: Your calendar_tool currently ignores the query and creates a fixed event.
    # You would need to add logic here to parse the query and create a dynamic event.
    result = calendar_tool.invoke(state["query"])
    return {"query": state["query"], "tool_choice": "calendar", "result": result}

def chat_response(state: ChatState) -> ChatState:
    print("Routing to Chat...") # Added for debugging
    # Use .invoke() with ChatGoogleGenerativeAI
    result = llm.invoke(state["query"]).content
    return {"query": state["query"], "tool_choice": "chat", "result": result}

# === LangGraph ===
graph_builder = StateGraph(ChatState)
graph_builder.set_entry_point("router")

graph_builder.add_node("router", route_tool)
graph_builder.add_node("search", use_search)
graph_builder.add_node("calendar", use_calendar)
graph_builder.add_node("chat", chat_response)
graph_builder.add_node("end", lambda x: x)

graph_builder.add_conditional_edges(
    "router",
    lambda state: state["tool_choice"],
    {
        "search": "search",
        "calendar": "calendar",
        "chat": "chat"
    }
)

graph_builder.add_edge("search", "end")
graph_builder.add_edge("calendar", "end")
graph_builder.add_edge("chat", "end")

graph = graph_builder.compile()

# === Chat Interface ===
def chatbot_interface(user_input):
    # Initial state before routing
    state = {"query": user_input, "tool_choice": "chat", "result": None} # 'chat' is a default, router will change it
    result = graph.invoke(state)
    return result["result"]

# === Interactive Loop ===
print("Chatbot started! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chatbot_interface(user_input)
    print(f"Bot: {response}")